# Agentes de Azure AI con soporte para el Protocolo de Contexto de Modelo (MCP)

Este cuaderno demuestra cómo usar los Agentes de Azure AI con herramientas del Protocolo de Contexto de Modelo (MCP) para crear un agente inteligente que pueda aprovechar servidores MCP externos para capacidades mejoradas.


## Instalar los paquetes de NuGet necesarios

Primero, necesitamos instalar el paquete Azure AI Agents Persistent, que proporciona la funcionalidad principal para trabajar con Azure AI Agents.


## Beneficios de la Autenticación Sin Claves

Este cuaderno demuestra la **autenticación sin claves**, que ofrece varias ventajas:
- ✅ **Sin claves de API que gestionar** - Utiliza autenticación basada en identidad de Azure
- ✅ **Mayor seguridad** - No se almacenan secretos en el código ni en la configuración
- ✅ **Rotación automática de credenciales** - Azure gestiona el ciclo de vida de las credenciales
- ✅ **Acceso basado en roles** - Utiliza Azure RBAC para permisos detallados

El `DefaultAzureCredential` usará automáticamente la mejor fuente de credenciales disponible:
1. Identidad Administrada (cuando se ejecuta en Azure)
2. Credenciales de Azure CLI (durante el desarrollo)
3. Credenciales de Visual Studio
4. Variables de entorno (si están configuradas)


In [1]:
#r "nuget: Azure.AI.Agents.Persistent, 1.1.0-beta.4"

Installed Packages Azure.AI.Agents.Persistent, 1.1.0-beta.4

Instala el paquete Azure Identity para la autenticación con servicios de Azure utilizando DefaultAzureCredential.


In [2]:
#r "nuget: Azure.Identity, 1.14.2"

Installed Packages Azure.Identity, 1.14.2

## Importar los Espacios de Nombres Necesarios

Importa los espacios de nombres necesarios para Azure AI Agents y Azure Identity.


In [3]:
using Azure.AI.Agents.Persistent;
using Azure.Identity;

## Configurar el Cliente de Azure AI Agent (Autenticación sin Clave)

Configura las variables de configuración y crea el PersistentAgentsClient utilizando **autenticación sin clave**:
- **projectEndpoint**: El punto de conexión del proyecto de Azure AI Foundry
- **modelDeploymentName**: El nombre del modelo de IA desplegado (GPT-4.1 nano)
- **mcpServerUrl**: La URL del servidor MCP (API de Microsoft Learn)
- **mcpServerLabel**: Una etiqueta para identificar el servidor MCP
- **DefaultAzureCredential**: Utiliza identidad administrada, Azure CLI u otras fuentes de credenciales (no se requieren claves de API)


In [ ]:
var projectEndpoint = "Your Azure AI Foundry Project Endpoint";
var modelDeploymentName = "Your Azure OpenAI Model Deployment Name";
var mcpServerUrl = "https://learn.microsoft.com/api/mcp";
var mcpServerLabel = "mslearn";
PersistentAgentsClient agentClient = new(projectEndpoint, new DefaultAzureCredential());

## Crear Definición de Herramienta MCP

Crea una definición de herramienta MCP que se conecte al servidor MCP de Microsoft Learn. Esto permitirá que el agente acceda al contenido y la documentación de Microsoft Learn.


In [ ]:
MCPToolDefinition mcpTool = new(mcpServerLabel, mcpServerUrl);

## Crear el Agente de IA

Crea un agente de IA persistente con el modelo y las herramientas MCP especificadas. El agente está configurado con:
- El modelo GPT-4.1 nano  
- Instrucciones para usar las herramientas MCP como asistencia  
- Acceso al servidor MCP de Microsoft Learn  


In [ ]:
PersistentAgent agent = await agentClient.Administration.CreateAgentAsync(
   model: modelDeploymentName,
   name: "my-learn-agent",
   instructions: "You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
   tools: [mcpTool]
   );

## Crear hilo y enviar mensaje

Crea un hilo de conversación y envía un mensaje al usuario preguntando sobre la diferencia entre Azure OpenAI y OpenAI. Esto pondrá a prueba la capacidad del agente para usar las herramientas MCP y proporcionar información precisa.


In [7]:
PersistentAgentThread thread = await agentClient.Threads.CreateThreadAsync();

// Create message to thread
PersistentThreadMessage message = await agentClient.Messages.CreateMessageAsync(
    thread.Id,
    MessageRole.User,
    "What's difference between Azure OpenAI and OpenAI?");

## Configurar recursos de la herramienta MCP (sin claves)

Configura los recursos de la herramienta MCP. Para un enfoque completamente sin claves, puedes eliminar los encabezados personalizados si el servidor MCP admite la autenticación basada en identidad de Azure. El ejemplo a continuación muestra cómo agregar encabezados si es necesario, pero para escenarios sin claves, estos pueden no ser requeridos.


In [ ]:
// Option 1: Completely keyless (if MCP server supports Azure identity)
MCPToolResource mcpToolResource = new(mcpServerLabel);
ToolResources toolResources = mcpToolResource.ToToolResources();

// Option 2: With custom headers (if still needed for specific MCP servers)
// MCPToolResource mcpToolResource = new(mcpServerLabel);
// mcpToolResource.UpdateHeader("Authorization", "Bearer <your-token>");
// ToolResources toolResources = mcpToolResource.ToToolResources();

## Iniciar Ejecución del Agente

Crea e inicia una ejecución para procesar el mensaje del usuario. El agente utilizará las herramientas y recursos configurados de MCP para generar una respuesta.


In [9]:
ThreadRun run = await agentClient.Runs.CreateRunAsync(thread, agent, toolResources);

## Supervisar la Ejecución y Gestionar Aprobaciones de Herramientas (Sin Clave)

Supervisa el estado de ejecución del agente y gestiona cualquier aprobación de herramientas requerida. Este ciclo:
1. Espera a que la ejecución se complete o requiera acción
2. Aprueba automáticamente las llamadas a herramientas MCP cuando sea necesario
3. Para la autenticación sin clave, es posible que no se necesiten encabezados si el servidor MCP admite identidad de Azure


In [ ]:
while (run.Status == RunStatus.Queued || run.Status == RunStatus.InProgress || run.Status == RunStatus.RequiresAction)
{
    await Task.Delay(TimeSpan.FromMilliseconds(1000));
    run = await agentClient.Runs.GetRunAsync(thread.Id, run.Id);

    if (run.Status == RunStatus.RequiresAction && run.RequiredAction is SubmitToolApprovalAction toolApprovalAction)
    {
        var toolApprovals = new List<ToolApproval>();
        foreach (var toolCall in toolApprovalAction.SubmitToolApproval.ToolCalls)
        {
            if (toolCall is RequiredMcpToolCall mcpToolCall)
            {
                Console.WriteLine($"Approving MCP tool call: {mcpToolCall.Name}");
                
                // Option 1: Keyless approval (no headers needed)
                toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true));
                
                // Option 2: With headers (if required by specific MCP server)
                // toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true)
                // {
                //     Headers = { ["Authorization"] = "Bearer <your-token>" }
                // });
            }
        }

        if (toolApprovals.Count > 0)
        {
            run = await agentClient.Runs.SubmitToolOutputsToRunAsync(thread.Id, run.Id, toolApprovals: toolApprovals);
        }
    }
}

Approving MCP tool call: microsoft_docs_search


## Mostrar Resultados de la Conversación

Recupera y muestra todos los mensajes del hilo, mostrando tanto la pregunta del usuario como la respuesta del agente. Los mensajes se muestran en orden cronológico con marcas de tiempo e indicadores de rol.


In [12]:
using Azure;

AsyncPageable<PersistentThreadMessage> messages = agentClient.Messages.GetMessagesAsync(
    threadId: thread.Id,
    order: ListSortOrder.Ascending
);

await foreach (PersistentThreadMessage threadMessage in messages)
{
    Console.Write($"{threadMessage.CreatedAt:yyyy-MM-dd HH:mm:ss} - {threadMessage.Role,10}: ");
    foreach (MessageContent contentItem in threadMessage.ContentItems)
    {
        if (contentItem is MessageTextContent textItem)
        {
            Console.Write(textItem.Text);
        }
        else if (contentItem is MessageImageFileContent imageFileItem)
        {
            Console.Write($"<image from ID: {imageFileItem.FileId}>");
        }
        Console.WriteLine();
    }
}

2025-07-16 06:39:43 -       user: What's difference between Azure OpenAI and OpenAI?
2025-07-16 06:39:51 -  assistant: The main difference between Azure OpenAI and OpenAI lies in their deployment, management, and integration options:

1. **Azure OpenAI**:
   - A cloud service offered through Microsoft Azure.
   - Provides access to OpenAI models with additional enterprise features like security, compliance, and scale.
   - Allows integration with other Azure services, enabling seamless use within existing Azure-based solutions.
   - Offers managed deployment, monitoring, and support within the Azure ecosystem.
   - Suitable for organizations looking for enterprise-grade security, compliance, and regional availability.

2. **OpenAI (OpenAI API)**:
   - A standalone API service provided directly by OpenAI.
   - Accessible via the OpenAI platform without the need for Azure.
   - Focused on providing GPT models, DALL-E, etc., primarily for developers and researchers.
   - Suitable for indi


---

**Descargo de responsabilidad**:  
Este documento ha sido traducido utilizando el servicio de traducción automática [Co-op Translator](https://github.com/Azure/co-op-translator). Si bien nos esforzamos por garantizar la precisión, tenga en cuenta que las traducciones automatizadas pueden contener errores o imprecisiones. El documento original en su idioma nativo debe considerarse la fuente autorizada. Para información crítica, se recomienda una traducción profesional realizada por humanos. No nos hacemos responsables de malentendidos o interpretaciones erróneas que puedan surgir del uso de esta traducción.
